The features that will be used are the words that can be found in the each document in the entire corpus. In order to extract all the features from the corpus, we first need to extract all the words in the corpus, the words will serve as a feature

The following are the libraries that will be used in this tutorial

In [1]:
#imports here
import re
import os
import json
import pandas
import numpy as np
from pprint import pprint

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import csv 

from numpy import genfromtxt


Create a class that will hold the Data about each Corpus. 
In the paper, they used 4 different corpus

1. Bare : Original Email Data
2. Lemm : Words in the emails are lemmatized
3. Stop : Stops words are removed from the emails
4. Lemm_stop : Words are lemmatized and stop words are removed from the emails

In [2]:
# create a class for the the Corpus data, it will store the total number of emails in the corpus, 
# along with the total number of spam and legit emails
class CorpusData: 
    corpusName = ""
    totalEmailCtr = 0
    spamEmailCtr = 0
    legitEmailCtr = 0
    legitEmails = []
    spamEmails = []

    def __init__(self, corpusName, totalEmailCtr, spamEmailCtr, legitEmailCtr, legitEmailsArr, spamEmailsArr):
        self.corpusName = corpusName
        self.totalEmailCtr = totalEmailCtr
        self.spamEmailCtr = spamEmailCtr
        self.legitEmailCtr = legitEmailCtr
        self.legitEmails = legitEmailsArr
        self.spamEmails = spamEmailsArr

# corpus list 
corpus = ["bare", "lemm","lemm_stop", "stop"]

The function `extractWords`, extracts the words in a textfle given its filepath as its parameter

`words =  [i for i in words if len(i) >= 4]` filters out the words that are less than 4 letters

`words = list(set(words))` Creates a set from the list of words extracted, and duplicate words are removed


The function `writeToFile`, write to text file given the filepath, and the data to write as its parameters

In [15]:
def extractWords(filepath):
    file = open(filepath, 'r')
    # .lower() returns a version with all upper case characters replaced with lower case characters.
    text = file.read().lower()
    file.close()
    # replaces anything that is not a lowercase letter, a space, or an apostrophe with a space:
    text = re.sub('[^a-z]+', " ", text)
    words = list(text.split())
    
     # remove duplicate words in the list
    words = list(set(words))
    # removes words that are less than 4 letters/characters
    words =  [i for i in words if len(i) >= 4] 
    return words;

# this functions write the list of tokens in the tokenWords parameter on the filepath passed on filename parameter 
def writeToFile(filename, tokenWords):
    f = open(filename, 'w')
    json.dump(tokenWords, f)
    f.close()

The code below traverses all of the documents that is under a given directory, we have 4 corpus, so we need to do this 4 times, one for each corpus. 

In [20]:
#create a list that will hold CorpusData Object
corpusDataList = []

# this will hold all the words in the corpus
wordList = []
#corpus list
corpus = ["bare", "lemm", "lemm_stop", "stop"]

#this regular expression will match all text files with this pattern (Files that has this pattern are the Legitimate Emails)
pattern = re.compile("\d+-\d+msg\d+.txt")

#for each corpus
for corp in corpus:
    legitEmailCTR = 0
    spamCTR = 0
    counter = 0  
    spamEmails = []
    legitEmails = []
    rootdir = "Emails/"
    rootdir = rootdir + corp
    
    #for each subdirectory in a corpus (folders - part 1 - 10)
    for subdir, dirs, files in os.walk(rootdir):
        #for each file in a folder
        for file in files:   
            tempList = []
            filepath =  subdir + os.sep + file
            #words are extracted from a file
            tempList = extractWords(filepath)
            #extracted words are added to the word list
            wordList.extend(tempList)
            
            #increment total email counter for this corpus
            counter = counter +1
            # create a string of all the words in a document (instead of a list)
            joinedStr = ' '.join(tempList)
            # if file is a legitimate email
            if pattern.match(file):    
                #add string/file to legitimate emails list
                legitEmails.append(joinedStr)
                #increment legitimate email counter
                legitEmailCTR =  legitEmailCTR + 1
            else: 
                #add string/file to spam emails list
                spamEmails.append(joinedStr)
                #increment spam email counter
                spamCTR = spamCTR + 1
    #Update word list to remove all duplicates words in the corpus (cause, a word might occur in multiple documents/emails)     
    wordList = list(set(wordList))     
    
#     #tokens/words extracted from the emails will be stored in a text file so it can be used for further analyzation
    pd1 = pandas.DataFrame(wordList)
    pd1.to_csv("Features/"+corp+"/"+corp+"_words.csv",  header=False,  index=False)
    pd2 = pandas.DataFrame(legitEmails)
    pd2.to_csv("Features/"+corp+"/"+corp+"_legitEmails.csv",  header=False,  index=False)
    pd3 = pandas.DataFrame(spamEmails)
    pd3.to_csv("Features/"+corp+"/"+corp+"_spamEmails.csv",  header=False,  index=False)

# create a class that will hold all relevant information about the Corpus    
    x = CorpusData(corp, counter, spamCTR, legitEmailCTR, legitEmails, spamEmails)
    corpusDataList.append(x)

Create a 2D matrix of term occurrence, x = terms, y = documents, 1 = term occurred in document then 0 otherwise, 

We will now populate the term document matrix, we can have two approaches in doing this. 

1. Build our own term document matrix populator by traversing each document in a corpus, and compare each word in a document to the term list, to get its index and mark it as 1 (term occurred in document)

The first approach will have the following steps:

1. iterate over all the documents/emails in the corpus
2. Each document/email extract the words
3. Compare each word in the document to the term list
4. tdm[term][document] = 1

**MAJOR DOWNSIDE**: this will take so much time to finish executing

In [5]:
#create document list
documentList = corpusDataList[0].legitEmails

#initialize document counter to 0 (first document)
docCtr = 0
print ("Starting")
#for each document
for doc in documentList:   
#     print ("---DOC---",doc)
    #for each word in a document
    for word in doc:
        # if word is in the wordlist
#         print ("---WORD---",word)
        if  word in wordList: 
            #get index of word in the word list
            b=wordList.index(word)
            print ("index:", b)
            #update tdm to 1 (1: word occured in document)
            tdm[b][docCtr] = 1
    #update document counter + 1, for the next document
    docCtr = docCtr + 1

print ("done")
print (tdm)

Starting
done


NameError: name 'tdm' is not defined

2: or we can use a sklearn function called count vectorizer, example of what `CountVectorizer` will do

given a list of Vocabulary (in our case, Term/Word List) it will create a term document matrix (array) with all of the terms mapped on the documents 

A sample of this library is shown in the next cell. 
1. *vocab*:  is the list of all the terms that will be used to tag the documents
2. *doc*: The list of documents

In [6]:
vocab = ['hot', 'cold', 'old']
#load all the term list here
cv = sklearn.feature_extraction.text.CountVectorizer(vocabulary=vocab)

doc = ['pease porridge hot', 'pease porridge cold', 'pease porridge in the pot', 'nine days old']
#load all the documents here
cv.fit_transform(doc).toarray()

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 1]])

Applying that to our data

In [21]:
#load the vocabulary /word list/ term list from file
wordList = open("Features/bare/bare_words.csv").read().splitlines()
print ("There are: ", len(wordList), " Words")

#load the documents here
rootDIR = "Features/"
corpus = "bare/bare"
spamMail = "_spamEmails.csv"
legitMail = "_legitEmails.csv"

#create a document list
spamDocumentList = []
legitDocumentList = []

file_name_spam = rootDIR+corpus+spamMail
file_name_legit = rootDIR+corpus+legitMail

with open(file_name_spam, 'r') as f:  
    reader = csv.reader(f)   
    #for each row in the file, append it the document list
    for row in reader:
        for doc in row:
            spamDocumentList.append(doc)

print ("The File that we are using for the documents is:", file_name_spam)
print ("There are: ", len(spamDocumentList), "Spam Documents")

with open(file_name_legit, 'r') as f:  
    reader = csv.reader(f)   
    #for each row in the file, append it the document list
    for row in reader:
        for doc in row:
            legitDocumentList.append(doc)

print ("The File that we are using for the documents is:", file_name_legit)
print ("There are: ", len(legitDocumentList), "Legitimate Documents")
print ("done")

There are:  47661  Words
The File that we are using for the documents is: Features/bare/bare_spamEmails.csv
There are:  304 Spam Documents
The File that we are using for the documents is: Features/bare/bare_legitEmails.csv
There are:  2211 Legitimate Documents
done


After we have, the terms and the documents loaded from the file. We can now start creating the term document matrix, and the term document matrix to file


term document matrix (tdm)
1. rows : documents
2. columns : terms/words

In [22]:
#use term list here
cv = sklearn.feature_extraction.text.CountVectorizer(vocabulary=wordList)

# -------------------- SPAM -----------------------
#use documents list here
tdmSpam = cv.fit_transform(spamDocumentList).toarray()
print ("There are: ", tdmSpam.size, " cells in the term document matrix")

#create a numpy array
aSpam = np.asarray(tdmSpam)
#save the Term Document Matrix to CSV (so we can access it later)
np.savetxt("Features/bare/spamTDM.csv", aSpam, delimiter=",")


# -------------------- LEGITIMATE -----------------------
#use documents list here
tdmLegit = cv.fit_transform(legitDocumentList).toarray()
print ("There are: ", tdmLegit.size, " cells in the term document matrix")

#create a numpy array
aLegit= np.asarray(tdmLegit)
#save the Term Document Matrix to CSV (so we can access it later)
np.savetxt("Features/bare/legitTDM.csv", aLegit, delimiter=",")
print("done")

There are:  14488944  cells in the term document matrix
There are:  105378471  cells in the term document matrix
done


Once we have the term document matrix, we can now start computing for the Mutual Information of each term in the corpus

\begin{align}
MI(X;C) =\sum_{x \epsilon {0,1}, c \epsilon {spam,legitimate}}^{ } P(X=x, C=c) \cdot log \frac{P(X=x, C=c)}{P(X=x)\cdot P(C=c)}
\end{align}


*P(X=x, C=c)* : Total number of documents the word `x` occurred in documents that are class `c`

*P(X=x)* : Total number of documents the word `x` occurred in the corpus

*P(C=c)* : Total number of documents the are class  `c` in the corpus

In [23]:
#load the spamTDM from file
spamTDM = genfromtxt('Features/bare/spamTDM.csv', delimiter=',')
#prints the size of the matrix, (number of documents, number of terms)
print("Spam TDM : ", spamTDM.shape)

spamTermOccurCount = (spamTDM != 0).sum(0)
print ("done")

#create a numpy array
spamTermCount = np.asarray(spamTermOccurCount)
#save the Term Document Matrix to CSV (so we can access it later)
np.savetxt("Features/bare/spamTermCount.csv", spamTermCount, delimiter=",")
print("File saved")

Spam TDM :  (304, 47661)
done


In [24]:
#load the legitTDM from file
legitTDM = genfromtxt('Features/bare/legitTDM.csv', delimiter=',')
#prints the size of the matrix, (number of documents, number of terms)
print("Legit TDM : ", legitTDM.shape)
legitTermOccurCount = (legitTDM != 0).sum(0)
print ("done")


#create a numpy array
legitTermCount = np.asarray(legitTermOccurCount)
#save the Term Document Matrix to CSV (so we can access it later)
np.savetxt("Features/bare/legitTermCount.csv", legitTermCount, delimiter=",")
print("File Saved")

Legit TDM :  (2211, 47661)
done


After all that pre-processing, now we can *finally* compute the mutual information score of the term on the entire corpus

In [50]:
import math

totalLegit = corpusDataList[0].legitEmailCtr
totalSpam = corpusDataList[0].spamEmailCtr
print (totalLegit, totalSpam)

def computeMI(termSpamCount, termLegitCount):
    totalTermCount = termSpamCount + termLegitCount
    try:
        classSpam = termSpamCount * math.log10(termSpamCount/(totalTermCount*totalSpam))
    except ValueError:
        classSpam = 0
    
    try:
        classLegit = termLegitCount * math.log10(termLegitCount/(totalTermCount*totalLegit)) 
    except ValueError:
        classLegit = 0
        
    return (classSpam + classLegit) 
    


legitTermCountArr = genfromtxt('Features/bare/legitTermCount.csv', delimiter=',')
spamTermCountArr = genfromtxt('Features/bare/spamTermCount.csv', delimiter=',')
mi = []

vfunc = np.vectorize(computeMI)
mi = vfunc(spamTermCountArr, legitTermCountArr)

final = np.vstack((spamTermCountArr,legitTermCountArr, mi)).T

#save the Term Document Matrix to CSV (so we can access it later)
np.savetxt("Features/bare/termMI.csv", final, delimiter=",")
print("File Saved")

print (final)

2211 304
File Saved
[[  2.           4.         -20.00270968]
 [  0.           2.          -6.68917749]
 [  0.           1.          -3.34458874]
 ..., 
 [  0.           5.         -16.72294371]
 [  0.           2.          -6.68917749]
 [  0.           3.         -10.03376623]]
